# EDA - Bases de données patient, consommant et hospitalisation

### Installation libraire requise

In [ ]:
!pip install pandas_profiling

### Import librairies

In [ ]:
import numpy as np
import os
import pandas as pd
from pandas_profiling import ProfileReport
from tqdm import tqdm
import warnings
warnings.simplefilter("ignore")

### Import des bases de données

In [ ]:
hospit=pd.read_csv("base_hospit.csv", sep=';')
patient=pd.read_csv("base_patient.csv", sep=';')
consommant=pd.read_csv("consommant.csv", sep=';')

### Function utile

In [ ]:
def count():
    print(f"Nb patients = {len(patient)}")
    print(f"Nb hospit = {len(hospit)}")
    print(f"Nb conso = {len(consommant)}")

### Conversion des dates en datetime

In [ ]:
hospit['EXE_SOI_DTD']=pd.to_datetime(hospit['EXE_SOI_DTD'], dayfirst=True)
hospit['SEJ_NBJ']=hospit['SEJ_NBJ'].fillna(np.nan).astype(float)
hospit['SEJ_NBJ']=hospit['SEJ_NBJ'].astype('Int64')

In [ ]:
consommant['datemax']=pd.to_datetime(consommant['datemax'], dayfirst=True)

In [ ]:
patient['date_h0']=pd.to_datetime(patient['date_h0'], dayfirst=True)
patient['dte_deces']=pd.to_datetime(patient['dte_deces'], dayfirst=True)

## Encodage des patients

In [ ]:
dict_patients = {}
for i in range(len(patient)):
    code = patient.CODE_PATIENT.iloc[i]
    dict_patients[code] = i

In [ ]:
patient['CODE_PATIENT_ENC'] = ['P' + str(i) for i in range(len(patient))]

In [ ]:
enc_conso = []
for i in range(len(consommant)):
    code = consommant.CODE_PATIENT.iloc[i]
    patient_n = dict_patients[code]
    enc_conso.append('P'+str(patient_n))
consommant['CODE_PATIENT_ENC'] = enc_conso

In [ ]:
enc_hospit = []
for i in range(len(hospit)):
    code = hospit.BEN_NIR_IDT.iloc[i]
    patient_n = dict_patients[code]
    enc_hospit.append('P' + str(patient_n))
hospit['BEN_NIR_IDT_ENC'] = enc_hospit

In [ ]:
patient=patient.drop(columns=['CODE_PATIENT'])
consommant=consommant.drop(columns=['CODE_PATIENT'])
hospit=hospit.drop(columns=['BEN_NIR_IDT'])

In [ ]:
import pickle
    
with open('dict_codes_patients.pickle', 'wb') as handle:
    pickle.dump(dict_patients, handle)

In [ ]:
patient=patient.rename(columns={'CODE_PATIENT_ENC':'CODE_PATIENT'})
consommant=consommant.rename(columns={'CODE_PATIENT_ENC':'CODE_PATIENT'})
hospit=hospit.rename(columns={'BEN_NIR_IDT_ENC':'BEN_NIR_IDT'})

## Suppression des patients dont la première hospitalisation est postérieure à 2016 ou antérieure à 2010

In [ ]:
patient=patient[(patient['date_h0']<'01-01-2016')&(patient['date_h0']>='01-01-2010')]

In [ ]:
patient['date_h0']=pd.to_datetime(patient['date_h0'], dayfirst=True)

In [ ]:
consommant = consommant[consommant['CODE_PATIENT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [ ]:
consommant.head()

In [ ]:
hospit.head()

In [ ]:
hospit = hospit[hospit['BEN_NIR_IDT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [ ]:
hospit.info()

In [ ]:
count()

## On conserve uniquement les mois de décès

In [ ]:
patient['dte_deces']=patient['dte_deces'].dt.to_period('M')

In [ ]:
patient.head()

In [ ]:
count()

## On conserve uniquement les patients majeurs au moment de leur première hospitalisation

#### Nombre de patients mineurs dans les données

In [ ]:
np.unique(((pd.to_datetime(patient.date_h0).apply(lambda x : x.year) - patient.y_nais) < 18), return_counts=True)

In [ ]:
patient = patient[pd.to_datetime(patient.date_h0).apply(lambda x : x.year) - patient.y_nais >= 18]

In [ ]:
consommant = consommant[consommant['CODE_PATIENT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [ ]:
hospit = hospit[hospit['BEN_NIR_IDT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [ ]:
patient

In [ ]:
count()

## On retire les patients ayant déjà connu une hospitalisation avant l'étude (pop=0)

In [ ]:
patient=patient[patient["pop"]==1].reset_index(drop=True)

In [ ]:
consommant = consommant[consommant['CODE_PATIENT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [ ]:
hospit = hospit[hospit['BEN_NIR_IDT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [ ]:
patient = patient.drop(columns=['pop'])

In [ ]:
count()

## On retire les patients sans consommation depuis plus de 5 ans, non déclarés décédés

In [ ]:
patient_non_décédés=patient[patient['dte_deces'].fillna(0)==0]["CODE_PATIENT"]

In [ ]:
patients_sans_conso=consommant[(consommant["CODE_PATIENT"].isin(patient_non_décédés))&(consommant['datemax']<'01-01-2014')]['CODE_PATIENT']

In [ ]:
patient = patient[~patient["CODE_PATIENT"].isin(patients_sans_conso)].reset_index(drop=True)

In [ ]:
consommant = consommant[consommant['CODE_PATIENT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [ ]:
hospit = hospit[hospit['BEN_NIR_IDT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [ ]:
count()

## Ajout des insuffisances cardiaques avec état de choc cardiogénique

In [ ]:
#Insuffisances cardiaques et états de choc circulatoire
#hospit[hospit['GRG_GHM'].str.startswith('05M09')]

#Insuffisances cardiaques et états de choc cardiogénique (code R57)
#hospit[(hospit['GRG_GHM'].str.startswith('05M09'))&(hospit['DGN_PAL'].str.startswith('R57'))]

#Ajout d'une colonne CHOC
hospit['CHOC']=hospit.index.isin(hospit[(hospit['GRG_GHM'].str.startswith('05M09'))&(hospit['DGN_PAL'].str.startswith('R57'))|(hospit['DGN_REL'].str.startswith('R57'))].index)
hospit['CHOC']=hospit['CHOC'].astype('int')

In [ ]:
hospit.CHOC.sum()

## Ajout des hémodialyses

In [ ]:
hospit['HEMOD']=hospit.index.isin(hospit[(hospit['DGN_PAL'].str.startswith('Z49'))].index)
hospit['HEMOD']=hospit['HEMOD'].astype('int')

In [ ]:
hospit['DGN_PAL'].str.startswith('Z49').sum()

## Ajout des chimiothérapies

In [ ]:
hospit['CHIMIO']=hospit.index.isin(hospit[(hospit['DGN_PAL'].str.startswith('Z511'))|(hospit['DGN_PAL'].str.startswith('Z512'))].index)
hospit['CHIMIO']=hospit['CHIMIO'].astype('int')

In [ ]:
hospit['DGN_PAL'].str.startswith('Z511').sum(), hospit['DGN_PAL'].str.startswith('Z512').sum()

## Ajout des radiothérapies

In [ ]:
hospit['Radiotherapie']=hospit.index.isin(hospit[(hospit['DGN_PAL'].str.startswith('Z510'))|(hospit['DGN_REL'].str.startswith('Z510'))].index)
hospit['Radiotherapie']=hospit['Radiotherapie'].astype('int')

In [ ]:
hospit.Radiotherapie.sum()

## Ajout des transfusions

In [ ]:
hospit['Transfusion']=hospit.index.isin(hospit[(hospit['GRG_GHM'].str.startswith('28Z14Z'))].index)
hospit['Transfusion']=hospit['Transfusion'].astype('int')

In [ ]:
hospit.Transfusion.sum()

## Statistiques descriptives sur les datasets

### Hospitalisation

In [ ]:
hospit.info()

In [ ]:
profile_hospit = ProfileReport(hospit, title="Hospit Profile Report")

In [ ]:
profile_hospit.to_file("profile_hospit.html")

### Consommant

In [ ]:
consommant.head()

In [ ]:
consommant.info()

In [ ]:
profile_conso = ProfileReport(consommant, title="Consommant Profile Report")

In [ ]:
profile_conso.to_file("profile_conso.html")

### Patient

In [ ]:
patient.head()

In [ ]:
patient.info()

In [ ]:
profile_patient = ProfileReport(patient, title="Patient Profile Report")

In [ ]:
profile_patient.to_file("profile_patient.html")

####

#### Nombres de patients et d'hospitalisations après processing

In [ ]:
#Nombre de patients
patient_init = pd.read_csv("base_patient.csv", sep=';')
print(f"Il y avait {len(patient_init)} patients initialement. Suite à l'ensemble des critères de rejet, il en reste {len(patient)}.")
print(f"On en a enlevé {len(patient_init) - len(patient)}.")

In [ ]:
hospit_init = pd.read_csv("base_hospit.csv", sep=';')
print(f"Il y avait {len(hospit_init)} hospitalisations initialement. Suite à l'ensemble des critères de rejet, il en reste {len(hospit)}.")
print(f"On en a enlevé {len(hospit_init) - len(hospit)}.")

In [ ]:
conso_init = pd.read_csv("consommant.csv", sep=';')
print(f"Il y avait {len(conso_init)} consommations initialement. Suite à l'ensemble des critères de rejet, il en reste {len(consommant)}.")
print(f"On en a enlevé {len(conso_init) - len(consommant)}.")

#### Nombre de patients dialysés

###### Chimiothérapie

In [ ]:
hospit_chimio = hospit.loc[hospit.CHIMIO == 1]
nb_patients_chimio = hospit_chimio.BEN_NIR_IDT.nunique()
print(f"Il y a {len(hospit_chimio)} hospitalisations pour des raisons de chimiothérapie.")
print(f"Cela concerne {nb_patients_chimio} patients.")

###### Hémodialyse

In [ ]:
hospit_hemod = hospit.loc[hospit.HEMOD == 1]
nb_patients_hemod = hospit_hemod.BEN_NIR_IDT.nunique()
print(f"Il y a {len(hospit_hemod)} hospitalisations pour des raisons d'hémodialyse.")
print(f"Cela concerne {nb_patients_hemod} patients.")

###### Radiothérapie

In [ ]:
hospit_radiotherapie = hospit.loc[hospit.Radiotherapie == 1]
nb_patients_radiotherapie = hospit_radiotherapie.BEN_NIR_IDT.nunique()
print(f"Il y a {len(hospit_radiotherapie)} hospitalisations pour des raisons de radiothérapie.")
print(f"Cela concerne {nb_patients_radiotherapie} patients.")

###### Transfusions

In [ ]:
hospit_transfusion = hospit.loc[hospit.Transfusion == 1]
nb_patients_transfusion = hospit_transfusion.BEN_NIR_IDT.nunique()
print(f"Il y a {len(hospit_transfusion)} hospitalisations pour des raisons de transfusion.")
print(f"Cela concerne {nb_patients_transfusion} patients.")

###### Hospitalisations autres

In [ ]:
hospit_non_chronique = hospit.loc[(hospit.HEMOD == 0) & (hospit.CHIMIO==0) & (hospit.Radiotherapie==0) & (hospit.Transfusion==0)]
nb_patients_non_chronique = hospit_non_chronique.BEN_NIR_IDT.nunique()
print(f"Il y a {len(hospit_non_chronique)} hospitalisations non chroniques.")
print(f"Cela concerne {nb_patients_non_chronique} patients.")
print("Tous les patients de la base ont au moins une hospitalisation qui n'est pas chronique.")
print(f"On a enlevé, en tout, {len(hospit_init) - len(hospit_non_chronique)} hospitalisations.")

#### Profiling sur les hospitalisations après processing et pas chroniques

In [ ]:
profile_hospit_non_chroniques = ProfileReport(hospit_non_chronique, title="Hospitalisations Non Chroniques")
profile_hospit_non_chroniques.to_file("profile_hospit_non_chr.html")

## Export des bases de données nettoyées

In [ ]:
hospit.to_csv("base_full_hospit_anonyme.csv", index=False)

In [ ]:
hospit_non_chronique.to_csv("base_hospit_non_chr_anonyme.csv", index=False)

In [ ]:
patient.to_csv("patient_anonyme.csv", index=False)

In [ ]:
consommant.to_csv("consommant_anonyme.csv", index=False)